# Data preparation
Data preparation for EEG classifier. It includes downsampling and time truncation. E.g. supposing we have multichannel timeseries of size [time x channels x trials]. Initially we have time interval e.g. [-500, 1000] ms, after truncation we have e.g. [200, 500] ms. After downsampling we have e.g. 80 Hz instead of 500 Hz. After all data will have a shape [trials x channels x time (sparse)]

In [3]:
import numpy as np
import scipy.io

In [1]:
def downsample(data, freq_old=500, freq_new=128):
    oldsize = data.shape[0]
    newsize = oldsize*freq_new//freq_old
    indices = [i for i in range(oldsize) for j in range(newsize)]
    densedata = data[indices]
    outdata = []
    for i in range(newsize):
        outdata.append(np.mean(densedata[i*oldsize:(i+1)*oldsize,...], axis=0, keepdims=True))
    outdata = reduce(lambda a,b: np.append(a,b,0), outdata)
    return outdata

def trunc(data, t0, t):
    return data[:,t0:t,:]


In [9]:
def load_data(*file_names):
    data = []
    y = np.array([])
    for f in file_names:
        dic = scipy.io.loadmat(f)
        X, label = dic['eegT'], 1 if 'eegT' in dic.keys() else dic['eegNT'], 0
        X = np.transpose(X, [2, 0, 1])
        data.append(X)
        y = np.hstack(y, np.ones(X.shape[0])*label)
    X = reduce(lambda a,b: np.append(a, b, 0), data)
    return X, y
        

In [5]:
data = scipy.io.loadmat('/home/moskaleona/alenadir/data/rawData/25/eegT.mat')['eegT']

In [8]:
data[:,:]

(750, 19, 236)